headachegenetics.org_sites_default_files_migraine_ihgc2021_gws_gwama_0.txt

In [9]:
# Import the libraries
from dotenv import load_dotenv
load_dotenv('../config/.env')
import sys
import os

# Add the path to the scripts folder and import the functions
sys.path.append("../scripts/")
from data_acquisition import load_csv_file, load_parquet_file

# Define the bucket name
bucket_name = 'migraine-pressure-6week-analysis'

# Define the file keys
url_cities = 'weather/cities.csv'
url_countries = 'weather/countries.csv'
url_daily_weather = 'weather/daily_weather.parquet'
url_migraine = 'health/adult22.csv'

# Load the data into dataframes
df_cities = load_csv_file(bucket_name,url_cities)
df_countries = load_csv_file(bucket_name,url_countries)
df_daily_weather = load_parquet_file(bucket_name,url_daily_weather)
df_migraine = load_csv_file(bucket_name,url_migraine)

# Display the dataframes
if df_cities is not None:
    print(df_cities.head())

if df_countries is not None:
    print(df_countries.head())

if df_daily_weather is not None:
    print(df_daily_weather.head())

if df_migraine is not None:
    print(df_migraine.head())


  station_id   city_name      country       state iso2 iso3   latitude  \
0      41515    Asadabad  Afghanistan       Kunar   AF  AFG  34.866000   
1      38954    Fayzabad  Afghanistan  Badakhshan   AF  AFG  37.129761   
2      41560   Jalalabad  Afghanistan   Nangarhar   AF  AFG  34.441527   
3      38947      Kunduz  Afghanistan      Kunduz   AF  AFG  36.727951   
4      38987  Qala i Naw  Afghanistan     Badghis   AF  AFG  34.983000   

   longitude  
0  71.150005  
1  70.579247  
2  70.436103  
3  68.872530  
4  63.133300  
          country     native_name iso2 iso3  population       area    capital  \
0     Afghanistan       افغانستان   AF  AFG  26023100.0   652230.0      Kabul   
1         Albania       Shqipëria   AL  ALB   2895947.0    28748.0     Tirana   
2         Algeria         الجزائر   DZ  DZA  38700000.0  2381741.0    Algiers   
3  American Samoa  American Samoa   AS  ASM     55519.0      199.0  Pago Pago   
4          Angola          Angola   AO  AGO  24383301.0  124